In [5]:
import pandas as pd
import CellLayers

In [6]:
exp_df = pd.read_csv('../Data/PBMC_exp.csv',index_col=[0])
meta_df = pd.read_csv('../Data/PBMC_meta.csv',index_col=[0])
mod_df = pd.read_csv('../Data/pbmc_modularity.csv',index_col=[0])
sil_df = pd.read_csv('../Data/pbmc_silhouette_scores.csv', index_col=[0])

In [7]:
sankey = CellLayers.run(exp_df, meta_df, modularity=mod_df, silhouette=sil_df, genes=['CD3E'])

In [8]:
sankey.show()